In [1]:
import pandas as pd
import numpy as np

login = pd.read_excel('Login.xlsx')
login.head()

,账户ID,事件名称,日期,时间,国家地区,城市,操作系统,省份,停留时间/s,购物数量,总购物次数
0,4478275,login,2022-12-26,20:49:00,中国,成都,Android,四川,2,0,103
1,4404603,login,2022-12-26,23:59:00,中国,NaN,Android,湖南,208,8,156
2,4512119,login,2022-12-27,00:00:00,中国,怀化,Android,湖南,269,3,79
3,4510605,login,2022-12-26,23:59:00,中国,NaN,Android,新疆,267,3,92
4,4439089,login,2022-12-26,23:59:00,中国,莆田,Android,福建,225,6,134


In [2]:
signup = pd.read_excel('Sign_Up.xlsx')
signup.head()

,账户ID,日期,时间,国家地区,城市,操作系统,省份
0,4502753,2022-12-22,04:51:00,中国,梅州,Android,广东
1,4502755,2022-12-22,05:16:00,中国,重庆,Android,重庆
2,4502738,2022-12-22,03:16:00,中国,乌鲁木齐,IOS,新疆
3,4502745,2022-12-22,04:10:00,中国,合肥,Android,安徽
4,4502756,2022-12-22,05:21:00,中国,咸宁,Android,湖北


In [3]:
signup = signup[signup['账户ID']!='-']
signup.sort_values(['日期','时间'], inplace=True)
signup = signup[~signup.duplicated('账户ID', keep='first')]

In [9]:
from copy import deepcopy
df = deepcopy(login)
n = 11
today = np.datetime64(df['日期'].max())
last_n = (today - np.timedelta64(n, 'D')).astype(object).strftime('%Y-%m-%d')
last_n

'2022-12-20'

In [10]:
df = df[df['日期']>last_n]
df['日期'] = df['日期'].astype(str)
days = df['日期'].unique().tolist()
days.sort()

In [11]:
new_user = []
for i in days:
    new_user.append(signup[signup['日期']==i]['账户ID'].unique())
num_new_user = [len(i) for i in new_user]
num_new_user

[642, 932, 607, 612, 710, 818, 1007, 704, 1610, 1072, 2078]

In [49]:
left = pd.concat([pd.DataFrame(days), pd.DataFrame(num_new_user)], axis=1)
left.columns = ['日期','新增用户数']
left

,日期,新增用户数
0,2022-12-21,642
1,2022-12-22,932
2,2022-12-23,607
3,2022-12-24,612
4,2022-12-25,710
5,2022-12-26,818
6,2022-12-27,1007
7,2022-12-28,704
8,2022-12-29,1610
9,2022-12-30,1072


In [50]:
import warnings
warnings.filterwarnings('ignore')

for i in range(0,n-1):
    left[f'{i+1}日后留存'] = [0]*(n)

for i in days:
    for j in range(1,n):
        user_today = new_user[days.index(i)]
        user_compare = df[df['日期'] == (np.datetime64(i)+np.timedelta64(j, 'D')).astype(object).strftime('%Y-%m-%d')]['账户ID'].unique()
        num = len([x for x in user_today if x in user_compare])
        if num:
            left[f'{j}日后留存'][days.index(i)] = num
left_ratio = deepcopy(left)
left = left.replace(0, '')[:-1]
left

,日期,新增用户数,1日后留存,2日后留存,3日后留存,4日后留存,5日后留存,6日后留存,7日后留存,8日后留存,9日后留存,10日后留存
0,2022-12-21,642,145,84,71,65,55,46,42,39,39,35
1,2022-12-22,932,436,367,337,325,329,190,185,191,190,
2,2022-12-23,607,161,91,71,63,52,47,48,43,,
3,2022-12-24,612,142,109,72,48,40,41,33,,,
4,2022-12-25,710,165,108,88,70,60,61,,,,
5,2022-12-26,818,185,120,173,132,128,,,,,
6,2022-12-27,1007,272,213,216,216,,,,,,
7,2022-12-28,704,199,72,59,,,,,,,
8,2022-12-29,1610,435,284,,,,,,,,
9,2022-12-30,1072,315,,,,,,,,,


In [51]:
for i in range(0,n-1):
    left_ratio[f'{i+1}日后留存率(%)'] = left_ratio[f'{i+1}日后留存']/left_ratio['新增用户数']*100
    left_ratio.drop(f'{i+1}日后留存', axis=1, inplace=True)
left_ratio = left_ratio.replace(0, '')[:-1]
left_ratio

,日期,新增用户数,1日后留存率(%),2日后留存率(%),3日后留存率(%),4日后留存率(%),5日后留存率(%),6日后留存率(%),7日后留存率(%),8日后留存率(%),9日后留存率(%),10日后留存率(%)
0,2022-12-21,642,22.58567,13.084112,11.05919,10.124611,8.566978,7.165109,6.542056,6.074766,6.074766,5.451713
1,2022-12-22,932,46.781116,39.377682,36.158798,34.871245,35.300429,20.386266,19.849785,20.493562,20.386266,
2,2022-12-23,607,26.523888,14.991763,11.69687,10.378913,8.566722,7.742998,7.907743,7.08402,,
3,2022-12-24,612,23.202614,17.810458,11.764706,7.843137,6.535948,6.699346,5.392157,,,
4,2022-12-25,710,23.239437,15.211268,12.394366,9.859155,8.450704,8.591549,,,,
5,2022-12-26,818,22.616137,14.669927,21.149144,16.136919,15.647922,,,,,
6,2022-12-27,1007,27.010924,21.151936,21.449851,21.449851,,,,,,
7,2022-12-28,704,28.267045,10.227273,8.380682,,,,,,,
8,2022-12-29,1610,27.018634,17.639752,,,,,,,,
9,2022-12-30,1072,29.384328,,,,,,,,,


In [52]:
left.to_csv('用户留存.csv', encoding='gbk')
left_ratio.to_csv('用户留存率.csv', encoding='gbk')

In [54]:
signup.to_csv('日新增用户.csv', encoding='gbk')